Basic Imports

In [1]:
import tweepy 
import time

Set up API through Tweepy, define api calls function

In [34]:
client = tweepy.Client(bearer_token = "*****************")


query = 'ukraine -is:retweet lang:en'


"""Tweets Fields requires public metrics to be inlcuded, wait time is in seconds to wait per api call, and conditional ensures
tweets are not repeated, returns 3 arrs with metrics indexes corresponding to tweet indexes"""
def return_api_fetch(q,calls, tweet_fields, max_results, wait_time = False, sort_order = "recency"):
    tweets = []
    shares_arr = []
    view_arr = []
    like_arr = []
    for _ in range(0,calls):
        tweets_api = client.search_recent_tweets(query=q, tweet_fields=["public_metrics",  'created_at'], max_results=100,sort_order = sort_order)

        for tweet in tweets_api.data:
            if tweet.text not in tweets:
                tweets.append(tweet.text)
                shares_arr.append(tweet.public_metrics["retweet_count"])
                view_arr.append(tweet.public_metrics["impression_count"])
                like_arr.append(tweet.public_metrics["like_count"])
        if wait_time:
            time.sleep(wait_time)
    return tweets, shares_arr, view_arr, like_arr
        
tweets,shares,views,likes = return_api_fetch(q = query, calls = 10, tweet_fields = ["public_metrics",  'created_at'], max_results = 100)

In [10]:
"""Basic Preprocessing function for tweets, removes hashtags as they interfere with KPE, stop words are also removed"""
import nltk
from nltk.corpus import stopwords
import re
stop_words = (stopwords.words('english'))


def pre_process_tweet(string):
    string = string.lower()
    string = re.sub("[/\()?;:,.*&^%$!1234567890-]", "", string)
    string = while_replace(string)
    lis = string.split()
    final = []
    user_count = 0
    for word in lis:
        if word[0] == "#":
            pass
        elif word[0] == "@":
            final.append(word)
        elif (word.count("h") >= 2 and word.count("a") >= 2) or "lol" in word:
            final.append("laugh")

        elif len(word) < 2 and word != "i" and word != "a":
                pass
        
        elif "http" in word:
            m = re.search('https?://([A-Za-z_0-9.-]+).*',word)
            if m:
                final.append(m.group(1))
            else:
                pass            
        else:
            if word not in stop_words:
                final.append(word)
    return final

"""Sub function to remove characters that appear more than 3 times or double spaces"""
def while_replace(tweet):
    while '  ' in tweet:
        tweet = tweet.replace('  ', ' ')
    new = ""
    for index in range(0,len(tweet)):
        char = tweet[index]
        if index == 0 or index == len(tweet)-1:
            new += char
        else:
            if tweet[index-1] == char and tweet[index+1] == char:
                pass
            else:
                
                new += char
        
    return new



In [11]:
"""Preprocesses Tweets"""
processed_tweets = []
for tweet in tweets:
    processed_tweets.append((pre_process_tweet(tweet)))

In [14]:
"""Defining nodes and connections for unweighted directional graph"""

nodes = []
for tweet in processed_tweets:
    for word in tweet:
        if word not in nodes:
            nodes.append( word)
bigrams = []
for tweet in processed_tweets:
    for i in range(len(tweet)-1):
        pair = [tweet[i], tweet[i+1]]
        if pair not in bigrams:
            bigrams.append(pair)

In [15]:
"""Creating Network"""

import networkx as nx
graph = nx.DiGraph()
graph.add_nodes_from(nodes)
graph.add_edges_from(bigrams)

In [16]:
"""Scoring and ordering nodes with betweenness centrality"""
node_scores_inbetween = nx.betweenness_centrality(graph)
sorted_node_scores_inbetween = dict(sorted(node_scores_inbetween.items(), key=lambda item: -item[1]))

In [18]:
"""Key phrase array creation code block, only phrases len >= 2 are kept, key words are top 100 ranked nodes"""
key_phrase = []
for tweet in processed_tweets:
    temp = []
    for word in tweet:
        tag = True
        if word in list(sorted_node_scores_inbetween.keys())[0:100]:
            temp.append(word)
        else:
            if len(temp) >= 2:
                if temp not in key_phrase:
                    key_phrase.append(temp)
                    temp = []
                else:
                    temp = []
            else:
                temp = []
    if len(temp) >= 2 and tag:
        if temp not in key_phrase:
            key_phrase.append(temp)

In [55]:
"""Code block for ranking of key_phrase by likes, returns array of ranking metric
#TODO move to function so that other metrics can be used easily"""

"""Sub function to simply test if key_phrase is in a processed tweet"""
def contains(small, big):
    for i in range(len(big)-len(small)+1):
        for j in range(len(small)):
            if big[i+j] != small[j]:
                break
        else:
            return i, i+len(small)
    return False

ranking = []
for key_phrase_arr in key_phrase:
    count = 0
    for tweet in processed_tweets:
        if contains(key_phrase_arr,tweet):
            count += (likes[(processed_tweets.index(tweet))])
    ranking.append(count)